<a href="https://colab.research.google.com/github/linnea-backgard/DataScienceProjectSpring23/blob/main/projekt_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Importera bibliotek
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Modeller
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split

# Metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.metrics import ConfusionMatrixDisplay

# SMOTE för oversampling
#from imblearn.over_sampling import SMOTE



In [2]:

from google.colab import files
 
 
uploaded = files.upload()

Saving diabetes2.csv to diabetes2.csv


In [7]:

import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['diabetes2.csv']))
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# Visar 50 översta rader för att upptäcka nollvärden också
df.head(50)
# Visar information om datasetet
df.info()
# Visar hur många rader och kolumner
df.shape
# Kollar hur många nollvärden det finns i varje kolumn.
(df == 0).sum()
# Kollar hur många nollvärden det finns i hela dataframen.
(df == 0).sum().sum()
# Är detta så många värden som finns i hela dataframen????
df.sum().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


(768, 9)

Kollar referensvärden - insulin: https://emedicine.medscape.com/article/2089224-overview
Referensvärden glukos
Adult - 74-106 mg/dL or 4.1-5.9 mmol/L, For elderly patients, the normal reference ranges are as follows [1] : 60-90 years - 82-115 mg/dL or 4.6-6.4 mmol/L >90 years - 75-121 mg/dL or 4.2-6.7 mmol/L

In [8]:
# Dela upp datan träning och test
X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [11]:
# Delar i tränings och testset med stratify för att behålla proportionerna mellan klasserna
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, stratify=y, random_state=42)

In [12]:
X_train.head(50)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
353,1,90,62,12,43,27.2,0.580,24
711,5,126,78,27,22,29.6,0.439,40
373,2,105,58,40,94,34.9,0.225,25
46,1,146,56,0,0,29.7,0.564,29
682,0,95,64,39,105,44.6,0.366,22
360,5,189,64,33,325,31.2,0.583,29
414,0,138,60,35,167,34.6,0.534,21
339,7,178,84,0,0,39.9,0.331,41
694,2,90,60,0,0,23.5,0.191,25
33,6,92,92,0,0,19.9,0.188,28


In [13]:
y_train

353    0
711    0
373    0
46     0
682    0
      ..
451    1
113    0
556    0
667    1
107    0
Name: Outcome, Length: 614, dtype: int64

In [14]:
# Gör en ny variabel som innehåller både X_train och y_train
train_df = pd.concat([X_train, y_train], axis=1)

train_df.head(50)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
353,1,90,62,12,43,27.2,0.580,24,0
711,5,126,78,27,22,29.6,0.439,40,0
373,2,105,58,40,94,34.9,0.225,25,0
46,1,146,56,0,0,29.7,0.564,29,0
682,0,95,64,39,105,44.6,0.366,22,0
360,5,189,64,33,325,31.2,0.583,29,1
414,0,138,60,35,167,34.6,0.534,21,1
339,7,178,84,0,0,39.9,0.331,41,1
694,2,90,60,0,0,23.5,0.191,25,0
33,6,92,92,0,0,19.9,0.188,28,0


In [18]:

# Gör lista som innehåller kolumnerna som har nollvärden.
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
df = train_df

# Funktion för att rökna ut olika kolumners medianer
def filter_no_zero_values_median(df, columns):
    
    medians_class_0 = []
    medians_class_1 = []
    
    for col in columns:
        not_zero_outcome_0 = df[(df[col] != 0) & (df['Outcome'] == 0)][col]
        not_zero_outcome_1 = df[(df[col] != 0) & (df['Outcome'] == 1)][col]
        median_outcome_0 = np.median(not_zero_outcome_0)
        median_outcome_1 = np.median(not_zero_outcome_1)
        medians_class_0.append(median_outcome_0)
        medians_class_1.append(median_outcome_1)
   
    return (medians_class_0, medians_class_1)

train_df_medians = filter_no_zero_values_median(df, columns)

print(train_df_medians)

# Utveckla funktionen till kanske dictionairy eller så det blir tydligae när man skriver ut

([108.0, 70.0, 27.0, 100.0, 30.1, 0.3415, 27.0], [141.5, 76.0, 33.0, 168.0, 34.349999999999994, 0.461, 36.0])


In [21]:
df = train_df
medians_class_0, medians_class_1 = filter_no_zero_values_median(df, columns)

def impute_with_median(df, columns, medians_class_0, medians_class_1):
    
    # Imputera medianvärdet där det finns nollvärden i df
    for col in columns:
        df.loc[(df[col] == 0) & (df['Outcome'] == 0), col] = medians_class_0[columns.index(col)]
        df.loc[(df[col] == 0) & (df['Outcome'] == 1), col] = medians_class_1[columns.index(col)]
    
    return (df)

df_imputed = impute_with_median(df, columns, train_df_medians[0], train_df_medians[1])

print(df_imputed.head(50))

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin    BMI  \
353            1     90.0             62             12       43  27.20   
711            5    126.0             78             27       22  29.60   
373            2    105.0             58             40       94  34.90   
46             1    146.0             56             27      100  29.70   
682            0     95.0             64             39      105  44.60   
360            5    189.0             64             33      325  31.20   
414            0    138.0             60             35      167  34.60   
339            7    178.0             84             33      168  39.90   
694            2     90.0             60             27      100  23.50   
33             6     92.0             92             27      100  19.90   
218            5     85.0             74             22      168  29.00   
705            6     80.0             80             36      100  39.80   
76             7     62.0

In [25]:
columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
df = df_imputed

medians_df_imputed = filter_no_zero_values_median(df, columns)

print(medians_df_imputed)

([108.0, 70.0, 27.0, 100.0, 30.1, 0.3415, 27.0], [141.5, 76.0, 33.0, 168.0, 34.349999999999994, 0.461, 36.0])


In [23]:
# Kollar hur många nollvärden det finns i varje kolumn.
(train_df == 0).sum()

Pregnancies                  85
Glucose                       0
BloodPressure                 0
SkinThickness                 0
Insulin                       0
BMI                           0
DiabetesPedigreeFunction      0
Age                           0
Outcome                     400
dtype: int64

In [34]:
X_train = df_imputed.drop('Outcome', axis=1)

train_df_imputed = X_train

In [39]:
# Create a SimpleImputer object with strategy='median'

imp_median = SimpleImputer(strategy='median')

print(imp_median)

# Fit the imputer on the training set

imp_median.fit(train_df)




# Transform the training set

test_df = X_test

test_df_imputed = pd.DataFrame(imp_median.transform(test_df), columns=train_df.columns)


# Print the median of all columns in the training set

for col in train_df_imputed.columns:
  print('Median {} value in the training set: {}'.format(col, train_df_imputed[col].median()))

SimpleImputer(strategy='median')
Median Pregnancies value in the training set: 3.0
Median Glucose value in the training set: 117.0
Median BloodPressure value in the training set: 72.0
Median SkinThickness value in the training set: 28.0
Median Insulin value in the training set: 100.0
Median BMI value in the training set: 32.4
Median DiabetesPedigreeFunction value in the training set: 0.3825
Median Age value in the training set: 29.0


In [40]:
scaler = StandardScaler()

scaled_X_train = scaler.fit_transform(X_train)

In [42]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
353,1,90.0,62,12,43,27.2,0.580,24
711,5,126.0,78,27,22,29.6,0.439,40
373,2,105.0,58,40,94,34.9,0.225,25
46,1,146.0,56,27,100,29.7,0.564,29
682,0,95.0,64,39,105,44.6,0.366,22


In [46]:

scaled_X_train

array([[-0.85135507, -1.05708598, -0.82841991, ..., -0.76616897,
         0.31079384, -0.79216928],
       [ 0.35657564,  0.14257662,  0.47455755, ..., -0.4144692 ,
        -0.11643851,  0.56103382],
       [-0.5493724 , -0.55722656, -1.15416427, ...,  0.36220112,
        -0.76486207, -0.70759409],
       ...,
       [-0.85135507, -0.82381825, -0.17693118, ...,  0.83113414,
        -0.78607218, -0.28471812],
       [ 1.86648903, -0.35728279, -0.17693118, ..., -0.7222065 ,
        -1.01938346,  0.56103382],
       [ 0.05459296,  0.74240792, -1.15416427, ..., -0.42912336,
        -0.57700104,  0.30730824]])

In [50]:
# SVM Kernels (hyperparameter optimization)
for k in ('linear', 'poly', 'rbf', 'sigmoid'):
    clf = svm.SVC(kernel=k)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    print(k,':', round(accuracy_score(y_train, y_pred), 4))

linear : 0.8127
poly : 0.8616
rbf : 0.8632
sigmoid : 0.3713


In [52]:
# Training a Support Vector Machine
clf_n = svm.SVC(kernel = 'rbf', C=1, gamma=0.1)
clf_n.fit(X_train, y_train)

SVC(C=1, gamma=0.1)

In [53]:
y_pred = clf.predict(X_train)

print('Accuracy score using imputed and scaled dataset: ', round(accuracy_score(y_train, y_pred), 4))



Accuracy score using imputed and scaled dataset:  0.3713


In [54]:
models = {}
# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()
# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression()
# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()
# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models['Naive Bayes'] = GaussianNB()
# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()

In [55]:
accuracy, precision, recall, F1, ROC_auc = {}, {}, {}, {}, {}
for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
       
    # Make predictions
    predictions = models[key].predict(X_train)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_train)
    precision[key] = precision_score(predictions, y_train)
    recall[key] = recall_score(predictions, y_train)
    F1[key] = f1_score(predictions, y_train)
    ROC_auc[key]= roc_auc_score(predictions, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
# Build the KNN model
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

# Train the model on the training set
knn.fit(X_train, y_train)

# Make predictions on the testing set
Y_pred = knn.predict(X_train)

# Calculate evaluation metrics
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)
roc_auc = roc_auc_score(y_train, y_pred)
total = accuracy + precision + recall + f1 + roc_auc

# Print the evaluation metrics with a header and the values under it
print('{:<15} {:<15}'.format('Metric', 'Value'))
print('{:<15} {:.2f}'.format('Accuracy', accuracy))
print('{:<15} {:.2f}'.format('Precision', precision))
print('{:<15} {:.2f}'.format('Recall', recall))
print('{:<15} {:.2f}'.format('F1 score', f1))
print('{:<15} {:.2f}'.format('ROC AUC score', roc_auc))
print('{:<15} {}'.format('Total', round(total, 2)))

Metric          Value          
Accuracy        0.37
Precision       0.04
Recall          0.03
F1 score        0.03
ROC AUC score   0.29
Total           0.77
